In [1]:
# Importing Required Packages.

In [2]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler
from textblob import TextBlob

In [3]:
#Twitter App Credentials

In [4]:
consumer_key = "p4I1wgl2npsNdBDshu378Fjkd"
consumer_secret = "ulwcITBcrpPnb9EkwTukr7ERxLStRNudValhVWC68vlT8PdgGE"
access_token = "3194988066-pW6mxUKNgjFvR4rNzuhbONp23wPRBVxDCop1S0g"
access_token_secret = "xR5oi8SxmOJDw0SJAL0qfBhEsCGwXZe6SAOyIca4biiyR"

In [5]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [6]:
query = "Covid cases in India"

In [7]:
# Searching tweets using search function
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x0000027DB9D291C8>, _json={'created_at': 'Mon Nov 09 09:23:00 +0000 2020', 'id': 1325730598531395584, 'id_str': '1325730598531395584', 'full_text': 'Delhi overtakes Maharashtra, Kerala as highest contributor to new Covid-19 cases https://t.co/4dufSV4bLg', 'truncated': False, 'display_text_range': [0, 104], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/4dufSV4bLg', 'expanded_url': 'http://toi.in/zYXYRa/a24gk', 'display_url': 'toi.in/zYXYRa/a24gk', 'indices': [81, 104]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 134758540, 'id_str': '134758540', 'name': 'The Times Of India', 'screen_name': 'timesofindia', 'location': '

In [8]:
# Creating a Dataframe for storing the Tweets.
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [9]:
# Searching Tweets and saving them in the DataFrame.
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, exclude = 'retweets', lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [10]:
stream(query , file_name='my_tweets')

In [11]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,"Delhi overtakes Maharashtra, Kerala as highest...",The Times Of India,611371,13215158,New Delhi,True,0,0,2020-11-09 09:23:00
1,LIVE: 2 BSF personnel among six new Covid-19 c...,Chanchinthar,12462,5050,"Serchhip, India",False,0,0,2020-11-09 09:21:34
2,"#Delhi overtakes #Maharashtra, #Kerala as high...",Business Insider India🇮🇳,50524,72529,"New Delhi, India",True,6,1,2020-11-09 09:15:00
3,Is it compulsory for a college to conduct exam...,Manesh,165,36,Visakhapatnam,False,0,0,2020-11-09 09:07:10
4,Get latest updates on COVID-19 India https://t...,ElegantJ BI,2131,1259,"Ahmedabad, Gujarat, India",False,0,0,2020-11-09 09:01:10


In [12]:
# Cleaning the Tweets.
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [13]:
# Analyzing the Tweets.
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [14]:
# Adding the Cleaned Tweets to the Dataframe.
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [15]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,"Delhi overtakes Maharashtra, Kerala as highest...",The Times Of India,611371,13215158,New Delhi,True,0,0,2020-11-09 09:23:00,Delhi overtakes Maharashtra Kerala as highest ...
1,LIVE: 2 BSF personnel among six new Covid-19 c...,Chanchinthar,12462,5050,"Serchhip, India",False,0,0,2020-11-09 09:21:34,LIVE 2 BSF personnel among six new Covid 19 ca...
2,"#Delhi overtakes #Maharashtra, #Kerala as high...",Business Insider India🇮🇳,50524,72529,"New Delhi, India",True,6,1,2020-11-09 09:15:00,overtakes as highest contributor to new cases ...
3,Is it compulsory for a college to conduct exam...,Manesh,165,36,Visakhapatnam,False,0,0,2020-11-09 09:07:10,Is it compulsory for a college to conduct exam...
4,Get latest updates on COVID-19 India https://t...,ElegantJ BI,2131,1259,"Ahmedabad, Gujarat, India",False,0,0,2020-11-09 09:01:10,Get latest updates on COVID 19 India Total cas...


In [16]:
# Checking the Sentiment of the Tweet.
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,"Delhi overtakes Maharashtra, Kerala as highest...",The Times Of India,611371,13215158,New Delhi,True,0,0,2020-11-09 09:23:00,Delhi overtakes Maharashtra Kerala as highest ...,Positive
1,LIVE: 2 BSF personnel among six new Covid-19 c...,Chanchinthar,12462,5050,"Serchhip, India",False,0,0,2020-11-09 09:21:34,LIVE 2 BSF personnel among six new Covid 19 ca...,Positive
2,"#Delhi overtakes #Maharashtra, #Kerala as high...",Business Insider India🇮🇳,50524,72529,"New Delhi, India",True,6,1,2020-11-09 09:15:00,overtakes as highest contributor to new cases ...,Positive
3,Is it compulsory for a college to conduct exam...,Manesh,165,36,Visakhapatnam,False,0,0,2020-11-09 09:07:10,Is it compulsory for a college to conduct exam...,Negative
4,Get latest updates on COVID-19 India https://t...,ElegantJ BI,2131,1259,"Ahmedabad, Gujarat, India",False,0,0,2020-11-09 09:01:10,Get latest updates on COVID 19 India Total cas...,Positive


In [17]:
# Checking the Particular Tweet.

In [18]:
n = 0
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 Delhi overtakes Maharashtra, Kerala as highest contributor to new Covid-19 cases https://t.co/4dufSV4bLg

Clean tweet:
 Delhi overtakes Maharashtra Kerala as highest contributor to new Covid 19 cases

Sentiment of the tweet:
 Positive


In [20]:
n = 888
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 @Knighteagle4 @shubh_ch @goforkj @runcaralisarun Nd for ur info there were already many cases of covid in India by end of January 2020...

Clean tweet:
 ch Nd for ur info there were already many cases of covid in India by end of January 2020

Sentiment of the tweet:
 Positive


In [21]:
print("Tweets with Positive Sentiment:",df[df.Sentiment == 'Positive'].shape[0])
print("Tweets with Neutral Sentiment:",df[df.Sentiment == 'Neutral'].shape[0])
print("Tweets with Negative Sentiment:",df[df.Sentiment == 'Negative'].shape[0])

Tweets with Positive Sentiment: 487
Tweets with Neutral Sentiment: 301
Tweets with Negative Sentiment: 212


In [22]:
#create a function that shows original and cleaned tweets
def generate_tweets(n):
    print("===============Original Tweet=============\n")
    print(df['Tweets'][n],"\n")
    print("===============Cleaned Tweet==============\n")
    print(df['clean_tweet'][n],"\n")

In [23]:
generate_tweets(250)

===============Original Tweet=============

India's Covid-19 cases breach 85-lakh mark, over 45,000 new cases in a day https://t.co/6TMRuzApCl 

===============Cleaned Tweet==============

India s Covid 19 cases breach 85 lakh mark over 45 000 new cases in a day 



In [24]:
df.columns

Index(['Tweets', 'User', 'User_statuses_count', 'user_followers',
       'User_location', 'User_verified', 'fav_count', 'rt_count', 'tweet_date',
       'clean_tweet', 'Sentiment'],
      dtype='object')